## Financial Markets - Practical Lesson 5

### Recap

* Basic Python
* Discount factor interpolation and Libor forward rates: we implemented functions then classes
* Last lesson we looked at Python modules and we implemented a class to store data defining an OIS contract, and for calculating its NPV

### This lesson

Now we're going to look at how extract a discount curve from OIS market data, via a process sometimes called *bootstrapping*. This is the ABC of financial mathematics, since you almost always need a discount curve to price any contract, especially if you're interested in its NPV. We're going to concentrate on EONIA swaps in order to build an EUR discount curve.

### Getting the data

The first problem is actually getting the data from somewhere, and this is not actually as simple as it sounds.

The issue is that the EONIA swap market is OTC and it's not straightforward to access it. Unlike (some) listed futures, where anyone with a retail brokerage account can view and apply realtime prices, to trade in the EONIA swap market you have to be a financial institution or at least a large company and have an agreement with a broker which operates in the market. One of the main brokers in the OIS market is [ICAP](http://www.icap.com/).

Though there exist some electronic platforms in which market participants post bids and offers and other participants can apply them, in practice a lot of trading is still done over "voice", i.e. by phone or more commonly over chat. For convenience, however, Bloomberg provides a service which displays indicative realtime prices as provided by a selection of relevant brokers.

<img src="images/icap_3.png">

It's possible to set up an Excel spreadsheet which acquires this data from Bloomberg in realtime (on a Bloomberg terminal). From this spreadsheet, it's easy to export the data into a Python file - I have done this and saved the data in a module called `ois_data.py`. We will use a data set extracted in this way, and derive from it the corresponding discount curve.

In [1]:
import ois_data

In [2]:
type(ois_data.quotes)

list

In [3]:
ois_data.quotes[0]

{'months': 1, 'rate': -0.35}

In [4]:
ois_data.quotes[-1]

{'months': 720, 'rate': 0.997}

In [5]:
ois_data.observation_date

datetime.date(2016, 11, 23)

### Building OIS instances

Last week's exercises included writing a function which, given a start date and a number of months, returns the list of payment dates of the corresponding OIS.

In [6]:
from dateutil.relativedelta import relativedelta

def generate_swap_dates(start_date, n_months):
    
    dates = []
    
    for n in range(0, n_months, 12):
        dates.append(start_date + relativedelta(months=n))
    
    dates.append(start_date + relativedelta(months=n_months))
    
    return dates

In [7]:
generate_swap_dates(ois_data.observation_date, 9)

[datetime.date(2016, 11, 23), datetime.date(2017, 8, 23)]

In [8]:
generate_swap_dates(ois_data.observation_date, 15)

[datetime.date(2016, 11, 23),
 datetime.date(2017, 11, 23),
 datetime.date(2018, 2, 23)]

In [9]:
generate_swap_dates(ois_data.observation_date, 24)

[datetime.date(2016, 11, 23),
 datetime.date(2017, 11, 23),
 datetime.date(2018, 11, 23)]

This function can then be used to build an OIS object based on the data contained in `ois_data`.

In [10]:
ois_data.quotes[12]

{'months': 15, 'rate': -0.35}

In [11]:
from finmarkets import OvernightIndexSwap

ois = OvernightIndexSwap(
    
    # notional
    1e6,
    
    # list of payment dates, including start date
    generate_swap_dates(
        ois_data.observation_date,
        15                       
    ),
    
    # fixed rate
    -0.35

)

ois.payment_dates[-1]

datetime.date(2018, 2, 23)

In [12]:
# remember we could use the npv method to calculate the OIS's npv
# only problem is we don't yet have a discount curve with which to evaluate it!

ois.npv(curve)

NameError: name 'curve' is not defined

### Bootstrapping

The general idea is to find the discount curve such that it prices correctly each OIS, or at least as well as possible, by minimizing the sum of the square NPVs:

$$
\def\MarketQuote{\mathrm{MarketQuote}}
\def\NPV{\mathrm{NPV}}
\def\ois{\mathrm{ois}}
\def\curve{\mathrm{curve}}
\min_{\mathrm{\curve}} \left\{ \sum_{i=1}^n \NPV(\ois_i, \curve)^2 \right\}
$$

This description of the problem does not, in theory, specify any constraints on the pillar dates of the discount curve. However, the pillar dates determine the dimensionality $n$ of the optimization problem - i.e. the number of unknown variables. A curve with $n$ pillar dates has $n$ pillar discount factors (note that the first discount factor with value date equal to the today date, is constrained to 1, so it doesn't count).

In practice, therefore, it makes sense to choose the pillar dates in such a way that there are exactly the right number of degrees of freedom in the optimization to match the data exactly. By choosing the pillar dates of the discount curve equal to set of expiry dates the swaps, we can find a vector of pillar discount factor that perfectly recover a zero NPV for every swap. The reason for this is that each market quote will determine exactly one 'free' discount factor which is not already determined by the other market quotes - this can be seen by considering the mathematical expression for calculating the fixed leg of the OIS swaps, and the way that the payment date schedules are constructed.

Therefore, once we've fixed $\vec d$ to be a vector of pillar dates equal to the expiry dates of the OIS swaps, and we use the notation $\vec x$ to represent the vector of pillar discount factors, then the problem becomes:

$$
\min_{\vec x} \left\{ \sum_{i=1}^n \NPV(\ois_i, \curve(\vec d, \vec x)) ^2 \right\}
$$

Cast in this way, the problem looks a bit more like a mathematical optimization problem. We just need to define the object function and run it through a numerical optimization routine.

In [1]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates
import ois_data

pillar_dates = [ois_data.observation_date]

swaps = []

for quote in ois_data.quotes:
    swap = OvernightIndexSwap(
        
        # notional - doesn't really matter what we put here
        1e6,
        
        # payment dates
        generate_swap_dates(
            ois_data.observation_date,
            quote['months']
        ),
        
        # the fixed rate
        0.01 * quote['rate']
        
    )
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

n_df_vector = len(pillar_dates)

In [2]:
type(pillar_dates), len(pillar_dates), pillar_dates[0], pillar_dates[-1]

(list, 34, datetime.date(2016, 11, 23), datetime.date(2076, 11, 23))

In [3]:
def objective_function(x):
    
    curve = DiscountCurve(
        
        # today date
        ois_data.observation_date,
        
        # pillar dates
        pillar_dates,
        
        # pillar discount factors
        x
        
    )
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq
        

In [4]:
from scipy.optimize import minimize

In [5]:
x0 = [1.0 for i in range(n_df_vector)]
bounds = [(0.01, 100.0) for i in range(n_df_vector)]

# we want to constrain the first pillar discount factor = 1 (because it has pillar date = today)
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [6]:
result

      fun: 0.0007370890117814888
 hess_inv: <34x34 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.40061365e+05, -4.14161280e+01, -1.93984741e+01,  5.37030079e+00,
        3.02530223e+01,  5.95243457e+01,  9.05547884e+01,  1.24363295e+02,
        1.59125629e+02,  1.97071574e+02,  2.36973096e+02,  2.76028231e+02,
       -9.72901535e+02, -3.95831915e+02, -3.67814737e+02, -3.29982597e+02,
       -3.11447882e+02,  1.31701062e+02,  5.96919251e+02,  9.85290168e+02,
        1.18797301e+03,  1.09656582e+03,  7.03858626e+02,  7.86777868e+01,
       -6.55082634e+02, -1.36397212e+03, -1.89121870e+02,  1.93487828e+03,
        5.40226051e+02, -1.65947564e+02, -5.36852034e+02, -2.47750882e+03,
       -1.84414691e+02,  1.90090790e+03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 875
      nit: 12
   status: 0
  success: True
        x: array([1.        , 1.00029175, 1.00058831, 1.00089012, 1.00116802,
       1.00147021, 1.00176786, 1.00207128, 1.00236508, 1.0

In [19]:
objective_function(x0)

1055841619695.9585

In [20]:
objective_function(result.x)

0.0007370932827914341

In [21]:
curve = DiscountCurve(ois_data.observation_date, pillar_dates, result.x)

from datetime import date
curve.df(date(2056, 11, 23))

0.643506361425295

In [22]:
import math
-math.log(curve.df(date(2056, 11, 23))) / 50

0.008816467326600488

In [25]:
list(result.x)

[1.0,
 1.0002917467402148,
 1.0005883131272417,
 1.0008901199538236,
 1.0011680243827714,
 1.0014702089411212,
 1.001767864893772,
 1.0020712764009907,
 1.0023650754871887,
 1.0026688489207509,
 1.0029728065322525,
 1.0032577961650622,
 1.0035612434176608,
 1.0044596767629976,
 1.0052998330195502,
 1.0061426892578045,
 1.0069306074274715,
 1.0090620121240119,
 1.0093198010276425,
 1.0071011202467379,
 1.001898602291577,
 0.99379503923610302,
 0.98332969774581891,
 0.97101000579044472,
 0.9572316430521306,
 0.94268860498062101,
 0.92772535369376719,
 0.88314868768977373,
 0.81781130464382112,
 0.76554845436522456,
 0.71988664072852804,
 0.64350636142529505,
 0.59281977672104502,
 0.54547323706318263]

In [26]:
pillar_dates

[datetime.date(2016, 11, 23),
 datetime.date(2016, 12, 23),
 datetime.date(2017, 1, 23),
 datetime.date(2017, 2, 23),
 datetime.date(2017, 3, 23),
 datetime.date(2017, 4, 23),
 datetime.date(2017, 5, 23),
 datetime.date(2017, 6, 23),
 datetime.date(2017, 7, 23),
 datetime.date(2017, 8, 23),
 datetime.date(2017, 9, 23),
 datetime.date(2017, 10, 23),
 datetime.date(2017, 11, 23),
 datetime.date(2018, 2, 23),
 datetime.date(2018, 5, 23),
 datetime.date(2018, 8, 23),
 datetime.date(2018, 11, 23),
 datetime.date(2019, 11, 23),
 datetime.date(2020, 11, 23),
 datetime.date(2021, 11, 23),
 datetime.date(2022, 11, 23),
 datetime.date(2023, 11, 23),
 datetime.date(2024, 11, 23),
 datetime.date(2025, 11, 23),
 datetime.date(2026, 11, 23),
 datetime.date(2027, 11, 23),
 datetime.date(2028, 11, 23),
 datetime.date(2031, 11, 23),
 datetime.date(2036, 11, 23),
 datetime.date(2041, 11, 23),
 datetime.date(2046, 11, 23),
 datetime.date(2056, 11, 23),
 datetime.date(2066, 11, 23),
 datetime.date(2076, 1